# BEAST Workflow Example

This notebook provides an example for how to use the Bayesian Extinction and Stellar Tool (BEAST). 

Before starting, please make sure you have at least BEAST v2.0 installed. 

If you have questions or comments about the content included below, please contact Christina Lindberg (christina.lindberg@live.com).

In [6]:
import h5py

import numpy as np
from astropy import wcs
from astropy.io import fits
from astropy.table import Table

import glob
import os
import types
import argparse
import importlib
from IPython.display import Image

import matplotlib.pyplot as plt
%matplotlib inline

from beast.plotting import (plot_mag_hist, 
                            plot_ast_histogram, 
                            plot_noisemodel)

from beast.tools.run import (
    create_physicsmodel,
    make_ast_inputs,
    create_obsmodel,
    run_fitting,
    merge_files,
    create_filenames,
)

from beast.physicsmodel.grid import SEDGrid
from beast.fitting import trim_grid
import beast.observationmodel.noisemodel.generic_noisemodel as noisemodel
from beast.observationmodel.observations import Observations
    
from beast.tools import (
     beast_settings,
     create_background_density_map,
     split_ast_input_file,
     split_catalog_using_map,
     cut_catalogs,
     setup_batch_beast_trim,
)

For this example, we will be walking through a standard BEAST workflow example using photometry from the Panchromatic Hubble Andromeda Treasury (PHAT) survey (Dalcanton et al. 2012). 

If you are running this example notebook locally, the example data can be fetched at https://www.dropbox.com/sh/91aefrp9gzdc9z0/AAC9Gc4KIRIB520g6a0uLLama?dl=0

You can download the files manually from the link above and place them in a folder called `/data` within the same folder as this notebook.

## Step 1a. Make magnitude histograms
The first thing we need to do is understand the range of stellar magnitudes we are working with in this data set.

To do this we can make histograms of all the magnitudes of all the stars in all the different filters from the photometric catalog. This is done so that we know where the peaks of the histograms are in each filter. These peaks will then be used later when we make source density maps.

Essentially what happens is that, for the density maps, we only count objects within a certain range, currently set to mag_cut = 15 - (peak_for_filter-0.5). So if the peak was 17.5, then the objects that would be counted would have to be in the range between 15 and 18.

The reason we only count brighter sources is because dimmer sources tend to not be properly observed, especially as the magnitudes near the telescope limit. There will always be far more dim sources than bright sources, but if we know how many bright sources there are, then we can extrapolate as to how many dim sources there should be, and probably get a better understand from that than if we were to try and actually count all the dim sources we detect.

**Variable Information**

- field_name : the string name of the main photometric catalog we are working with. This variable will be used to rename a lot of different files in the future which is why we have it as a separate variable.
- gst_file : stands for good-stars, this is the full name for the original photometric catalog we are working with.

In [2]:
field_name = "M31-B09-EAST_chunk"
gst_file = "./data/%s.st.fits" %field_name

We can see what type of data this fits file holds by making a table. There should be around 50,000 sources in this calalog, which is quite small compared to the original file.

Note: st stands for stars. We also sometimes name things gst for good stars to signify when cuts have been made.

In [3]:
hdul = fits.open(gst_file)
Table(hdul[1].data)

F814W_ST,F814W_GST,F475W_ST,F475W_GST,F275W_ST,F275W_GST,F160W_ST,F160W_GST,F336W_ST,F336W_GST,F110W_ST,F110W_GST,X,Y,RA,DEC,CHI_GL,SNR_GL,SHARP_GL,ROUND_GL,CROWD_GL,F475W_COUNT,F475W_SKY,F475W_RATE,F475W_RATERR,F475W_VEGA,F475W_TRANS,F475W_ERR,F475W_CHI,F475W_SNR,F475W_SHARP,F475W_ROUND,F475W_CROWD,F814W_COUNT,F814W_SKY,F814W_RATE,F814W_RATERR,F814W_VEGA,F814W_TRANS,F814W_ERR,F814W_CHI,F814W_SNR,F814W_SHARP,F814W_ROUND,F814W_CROWD,F275W_COUNT,F275W_SKY,F275W_RATE,F275W_RATERR,F275W_VEGA,F275W_TRANS,F275W_ERR,F275W_CHI,F275W_SNR,F275W_SHARP,F275W_ROUND,F275W_CROWD,F336W_COUNT,F336W_SKY,F336W_RATE,F336W_RATERR,F336W_VEGA,F336W_TRANS,F336W_ERR,F336W_CHI,F336W_SNR,F336W_SHARP,F336W_ROUND,F336W_CROWD,F110W_COUNT,F110W_SKY,F110W_RATE,F110W_RATERR,F110W_VEGA,F110W_TRANS,F110W_ERR,F110W_CHI,F110W_SNR,F110W_SHARP,F110W_ROUND,F110W_CROWD,F160W_COUNT,F160W_SKY,F160W_RATE,F160W_RATERR,F160W_VEGA,F160W_TRANS,F160W_ERR,F160W_CHI,F160W_SNR,F160W_SHARP,F160W_ROUND,F160W_CROWD,EXT,CHIP,MAJAX_GL,OBJTYPE_GL,F475W_FLAG,F814W_FLAG,F275W_FLAG,F336W_FLAG,F110W_FLAG,F160W_FLAG
bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64
True,True,True,True,True,True,True,True,True,True,True,True,869.8900000046692,44.40999999483233,11.131832570656686,41.603419438518735,4.03,2917.0,-0.03,0.02,0.251,233000000.0,87358.19,9.039999999999999e-08,1.3800000000000002e-09,17.61,17.721,0.017,2.48,65.4,0.005,0.038,0.175,198000000.0,102360.6,1.59e-07,1.27e-09,16.996,16.986,0.009000000000000001,1.77,124.8,0.01,0.049,0.22699999999999998,5970000.0,2256.98,1.25e-07,1.16e-09,17.255,nan,0.01,2.92,107.7,-0.038,0.018000000000000002,0.086,23800000.0,7534.45,1.69e-07,6.92e-10,16.932000000000002,nan,0.004,2.63,243.9,-0.031,0.059000000000000004,0.084,79700000.0,88039.23,1.6e-07,2.58e-10,16.992,nan,0.002,2.23,619.1,-0.004,0.1,0.35,56000000.0,93657.59,1.8e-07,2.53e-10,16.864,nan,0.002,1.51,708.8,-0.018000000000000002,0.027000000000000003,0.396,0,1,165,1,2,2,0,0,0,0
True,True,True,True,True,True,True,True,True,True,True,True,868.3900000040028,42.189999993549804,11.13180414033402,41.60338891056161,4.13,2566.5,-0.05,0.028999999999999998,0.27699999999999997,125000000.0,46004.84,4.85e-08,2.51e-10,18.285999999999998,18.412,0.006,3.02,193.3,-0.053,-0.006,0.231,118000000.0,77080.48,9.23e-08,1.24e-09,17.587,17.577,0.015,5.87,74.3,-0.096,0.044000000000000004,0.324,4360000.0,2332.74,9.15e-08,7.099999999999999e-10,17.597,nan,0.008,2.08,128.8,-0.057,0.008,0.092,16500000.0,8339.99,1.1699999999999999e-07,9.25e-10,17.329,nan,0.009000000000000001,3.74,126.5,-0.085,-0.006999999999999999,0.106,49500000.0,82165.18,9.92e-08,2.82e-10,17.508,nan,0.003,2.79,352.5,0.013000000000000001,-0.079,0.516,35100000.0,87515.55,1.13e-07,3.31e-10,17.371,nan,0.003,2.18,340.0,-0.026000000000000002,-0.004,0.635,0,1,75,1,2,2,0,2,0,0
True,True,False,False,False,False,True,True,False,False,True,True,297.1500000049341,22.169999994536283,11.135275566985227,41.595888643103535,1.17,1663.1,-0.08800000000000001,0.061,0.008,29910.7,10270.95,1.1499999999999999e-11,1.34e-12,27.348000000000003,27.805999999999997,0.126,1.34,8.6,-0.634,0.177,0.025,4160000.0,21190.25,3.17e-09,1.0000000000000001e-11,21.248,21.423000000000002,0.003,0.9,316.3,-0.063,0.049,0.001,-1.7,44.5,-6.900000000000001e-14,1.49e-11,nan,nan,9.999,0.87,-0.0,1.058,2.951,0.0

As we can see, there's a lot of columns and even more rows. For plotting the magnitude histograms, we're going to be interested in any column that contains the name VEGA. These are the columns with the magnitudes for each filter.

We could also use the X and Y columns to plot where are the sources are located, or the RA and DEC to map their actual position in the sky.

In larger projects we might have multiple fields to analyze during each run, so there would be multiple field_names. Since this is just a small example, we just have one field so our index will always be equal to 0.

In [4]:
# the list of fields (we only have 1 for this example.)
field_names = [field_name]

Now we can create some histogram plots to visualize the magnitude distribution of our sources.



In [10]:
# this 'if' statement just checks if there's already a histogram file
image_file = './data/'+field_names[0]+'.st_maghist.pdf'
if not os.path.isfile(image_file):
    peak_mags = plot_mag_hist.plot_mag_hist(gst_file, stars_per_bin=70, max_bins=75)

FileNotFoundError: [Errno 2] No such file or directory: './data/M31-B09-EAST_chunk.st.fits'

You can check out the results for the histograms in the file ending with _maghist.pdf

In [9]:
![title](./data/'+field_names[0]+'.st_maghist.pdf)

zsh:1: number expected
